In [1]:
from helper_utils import word_wrap

In [2]:
from pypdf import PdfReader

# List of PDF file names
pdf_files = [
    "Stockology Article 1.pdf",
]

# Dictionary to store text content from each PDF
pdf_texts = {}

# Loop through each PDF and extract text
for file in pdf_files:
    reader = PdfReader(file)
    text_content = [p.extract_text().strip() for p in reader.pages if p.extract_text()]
    
    # Store text content for each PDF by filename
    pdf_texts[file] = text_content

# Display the first page of each PDF's text, for example
for file, text_content in pdf_texts.items():
    print(f"--- Content from {file} ---")
    print(word_wrap(text_content[0]) if text_content else "No text found")
    print("\n")


--- Content from Stockology Article 1.pdf ---
Stockology: Textile, fertiliser and food business will 
attract big
ticket investments 
MAHESH  GOWANDE  AUGUST 11, 2024 / 07:46 IST  
The
long-term structure is very much intact, and nothing has changed.
Weekly and monthly stops 
and levels are still same and there is no
change. 
 
Review 
As expected, the markets opened with a gap down and
tried to recover by higher opening on 
Tuesday. They failed and closed
lower than Monday. Most PSU's remained in the limelight. 
Hammering and
recovery were witnessed in the defence stocks while real strength was
seen in the 
speciality chemical space along with FMCG companies. A
clear shifting of sector focus, and stock 
churning is visible when
looking at the past week's data. 
 
Technical 
Last week, we
highlighted that markets were showing signs of tiredness and weakness,
but TimeMap 
was positive, and markets remained in the optimistic zone.
Now, since the Sun entered Nakshatra 
Ashlesha, TimeMap i

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter

In [4]:
# Join the texts from all PDFs into one combined string
combined_text = '\n\n'.join(['\n\n'.join(text) for text in pdf_texts.values()])

# Split the combined text into chunks
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text(combined_text)

# Check if there are enough chunks before accessing character_split_texts[10]
if len(character_split_texts) > 10:
    print(word_wrap(character_split_texts[10]))
else:
    print("Not enough chunks to display character_split_texts[10].")

print(f"\nTotal chunks: {len(character_split_texts)}")


Not enough chunks to display character_split_texts[10].

Total chunks: 7


In [5]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([character_split_texts[2]]))

f:\PDF Q&A (Telecome Industries) Using Google Gemini\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


[array([ 7.38073364e-02,  9.88668799e-02,  2.76262667e-02,  9.54067409e-02,
        1.01144627e-01, -1.85774159e-04, -7.82095827e-03, -4.31412980e-02,
       -2.53293328e-02, -5.84532460e-03, -2.33445130e-02,  1.90906394e-02,
        1.41973468e-02, -1.94562189e-02,  7.18784109e-02,  3.20245065e-02,
       -2.11714003e-02, -8.03388357e-02, -3.85510027e-02, -3.54274102e-02,
       -6.44518510e-02, -9.91385989e-03, -5.58329001e-02, -4.28174734e-02,
       -4.15197983e-02, -5.05984239e-02,  7.35668372e-03, -8.09432119e-02,
       -2.08313856e-02, -7.17076063e-02, -2.31249835e-02,  3.59964818e-02,
       -1.33877888e-01,  4.77259746e-03,  2.55596209e-02,  1.72578916e-02,
        4.75292206e-02, -6.79488406e-02,  6.01705685e-02, -6.14483282e-02,
        1.49324253e-01, -9.46486890e-02, -4.27764170e-02, -7.58237988e-02,
       -1.11332787e-02,  4.76106862e-03,  1.46574214e-01,  9.94465221e-03,
       -8.13697428e-02,  1.51600791e-02, -2.44525727e-02, -9.64657590e-03,
       -2.64489688e-02, 

In [6]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("Stockology_Article_1", embedding_function=embedding_function)

ids = [str(i) for i in range(len(character_split_texts))]

chroma_collection.add(ids=ids, documents=character_split_texts)
chroma_collection.count()

7

In [7]:
query = "What was the view on textile sector?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(word_wrap(document))
    print('\n')

Stockology: Textile, fertiliser and food business will 
attract big
ticket investments 
MAHESH  GOWANDE  AUGUST 11, 2024 / 07:46 IST  
The
long-term structure is very much intact, and nothing has changed.
Weekly and monthly stops 
and levels are still same and there is no
change. 
 
Review 
As expected, the markets opened with a gap down and
tried to recover by higher opening on 
Tuesday. They failed and closed
lower than Monday. Most PSU's remained in the limelight. 
Hammering and
recovery were witnessed in the defence stocks while real strength was
seen in the 
speciality chemical space along with FMCG companies. A
clear shifting of sector focus, and stock 
churning is visible when
looking at the past week's data. 
 
Technical 
Last week, we
highlighted that markets were showing signs of tiredness and weakness,
but TimeMap 
was positive, and markets remained in the optimistic zone.
Now, since the Sun entered Nakshatra


Ashlesha, TimeMap is shifting slightly negative and will get mor

In [8]:
from dotenv import load_dotenv
load_dotenv()

import os
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-pro")

In [9]:
from dotenv import load_dotenv
load_dotenv()

import os
import google.generativeai as genai

# Configure the Google Gemini API key
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Initialize the model
model = genai.GenerativeModel("gemini-1.5-flash")

# Prepare the information from retrieved documents
information = "\n\n".join(retrieved_documents)

# Construct the prompt for the model
prompt = (
    "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report. "
    "You will be shown the user's question, and the relevant information from the annual report. "
    f"Question: {query}.\n Information: {information}"
)

# Generate the response using the prompt
response = model.generate_content(prompt)  # Use the prompt string instead of messages

# Print the generated response
print(response.text)

The article mentions that the textile sector, along with fertilizer and food businesses, will attract a lot of big-ticket investments. This suggests a positive outlook for the textile sector. 

